In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# reading mnist using libraries shipped with tf
mnist = input_data.read_data_sets("../data/mnist/", one_hot=True) 

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [3]:
mnist.__dict__['train'].__dict__.keys()

['_labels', '_epochs_completed', '_index_in_epoch', '_num_examples', '_images']

In [4]:
print len(mnist.train._labels)
print mnist.train.labels[0].shape
print len(mnist.train._images)
print mnist.train.images[0].shape

55000
(10,)
55000
(784,)


In [5]:
learning_rate = 0.5
number_of_epochs = 200
batch_size = 50

In [ ]:
#tf model building

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
W = tf.Variable(tf.random_normal([784, 10]))
b = tf.Variable(tf.zeros([1, 10]))

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

X_image = tf.reshape(X, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy_loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)

count_correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(y_conv,1))
accuracy = tf.reduce_mean(tf.cast(count_correct_prediction, tf.float32))

init_op = tf.initialize_all_variables()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op)
    for epoch in xrange(number_of_epochs):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.5})

    print "Training Accuracy = "+str(sess.run(accuracy, feed_dict={X: mnist.train.images, Y: mnist.train.labels, 
                                                                   keep_prob: 1.0}))
    print "Testing Accuracy = "+str(sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, 
                                                                 keep_prob: 1.0}))